# Museum Analysis

This notebook connects to the museum database and conducts simple analysis of the data.

## Imports

In [5]:
import psycopg2
import psycopg2.extras
import logging
from dotenv import dotenv_values

## Setup

In [6]:
logging.basicConfig(level=logging.DEBUG,
                    format='%(levelname)s: %(message)s')


def get_connection():
    """Establishes and returns a connection to the PostgreSQL database."""
    db_config = dotenv_values(".env")
    try:
        conn = psycopg2.connect(
            database=db_config['DB_NAME'],
            user=db_config['DB_USERNAME'],
            password=db_config['DB_PASSWORD'],
            host=db_config['DB_HOST'],
            port=db_config['DB_PORT']
        )
        logging.info("Database connection established.")

        return conn
    except Exception as e:
        logging.error("Error connecting to the database: %s", str(e))
        return None


def get_cursor(conn):
    """Returns a cursor for the provided connection."""
    return conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor)


conn = get_connection()
cursor = get_cursor(conn)

INFO: Database connection established.


## Queries

What exhibition is most frequently visited?

In [7]:
query = """
SELECT exhibition.exhibition_name, 
COUNT(request_interaction.exhibition_id) AS visit_count
FROM request_interaction
JOIN exhibition ON request_interaction.exhibition_id = exhibition.exhibition_id
GROUP BY exhibition.exhibition_name
ORDER BY visit_count DESC
LIMIT 1;
"""
cursor.execute(query)

result = cursor.fetchone()

print(f"Most Frequently Visited Exhibition: {result['exhibition_name']}")
print(f"Visit Count: {result['visit_count']}")

Most Frequently Visited Exhibition: Cetacean Sensations
Visit Count: 29


What hour of the day has the most ratings?

In [8]:
query = """
SELECT EXTRACT(HOUR FROM event_at) AS hour, COUNT(rating_interaction.rating_id) AS rating_count
FROM rating_interaction
GROUP BY hour
ORDER BY rating_count DESC
LIMIT 1;
"""

cursor.execute(query)
result = cursor.fetchone()

print(f"Hour with the Most Ratings: {result['hour']}:00")
print(f"Number of Ratings: {result['rating_count']}")

Hour with the Most Ratings: 10:00
Number of Ratings: 232


Exhibition with the Most Emergencies

In [9]:
query = """
SELECT exhibition.exhibition_name, 
COUNT(request_interaction.request_id) AS emergency_count
FROM request_interaction
JOIN request ON request_interaction.request_id = request.request_id
JOIN exhibition ON request_interaction.exhibition_id = exhibition.exhibition_id
WHERE request.request_value = 1
GROUP BY exhibition.exhibition_name
ORDER BY emergency_count DESC
LIMIT 1;
"""
cursor.execute(query)
result = cursor.fetchone()


print(f"Exhibition with the Most Emergencies: {result['exhibition_name']}")
print(f"Emergency Count: {result['emergency_count']}")

Exhibition with the Most Emergencies: The Crenshaw Collection
Emergency Count: 2


Average Rating for Each Exhibition

In [10]:
query = """
SELECT exhibition.exhibition_name, AVG(rating.rating_value) AS average_rating
FROM rating_interaction
JOIN rating ON rating_interaction.rating_id = rating.rating_id
JOIN exhibition ON rating_interaction.exhibition_id = exhibition.exhibition_id
GROUP BY exhibition.exhibition_name
ORDER BY average_rating DESC;
"""
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(f"""Exhibition: {row['exhibition_name']}, Average Rating: {
          row['average_rating']}""")

Exhibition: Cetacean Sensations, Average Rating: 2.8305084745762712
Exhibition: Adaptation, Average Rating: 1.9294117647058824
Exhibition: Measureless to Man, Average Rating: 1.9169435215946844
Exhibition: The Crenshaw Collection, Average Rating: 1.4259740259740260
Exhibition: Our Polluted World, Average Rating: 1.2162162162162162


Are positive ratings more frequent before or after 1 pm?

In [11]:
query = """
SELECT 
    CASE 
        WHEN EXTRACT(HOUR FROM event_at) < 13 THEN 'Before 1pm'
        ELSE 'After 1pm'
    END AS time_period,
    COUNT(*) AS rating_count
FROM rating_interaction
JOIN rating ON rating_interaction.rating_id = rating.rating_id
WHERE rating.rating_value >= 3
GROUP BY time_period
ORDER BY rating_count DESC;
"""

cursor.execute(query)
results = cursor.fetchall()

for result in results:
    print(f"""Time Period: {
          result['time_period']} - Positive Rating Count: {result['rating_count']}""")

Time Period: After 1pm - Positive Rating Count: 344
Time Period: Before 1pm - Positive Rating Count: 313


Do Zoology exhibitions get better ratings than other types?

In [12]:
query = """
SELECT department.department_name, AVG(rating.rating_value) AS avg_rating
FROM rating_interaction
JOIN exhibition ON rating_interaction.exhibition_id = exhibition.exhibition_id
JOIN rating ON rating_interaction.rating_id = rating.rating_id
JOIN department ON exhibition.department_id = department.department_id
GROUP BY department.department_name
ORDER BY avg_rating DESC;
"""

cursor.execute(query)
results = cursor.fetchall()

for result in results:
    print(f"""Department: {result['department_name']
                           } - Average Rating: {result['avg_rating']:.2f}""")

Department: Zoology - Average Rating: 2.20
Department: Entomology - Average Rating: 1.93
Department: Geology - Average Rating: 1.92
Department: Ecology - Average Rating: 1.22
